In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from natsort import natsorted

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
left_K = np.array(
    [[1000.0, 0.0, 360.0], [0.0, 1000.0, 640.0], [0.0, 0.0, 1.0]], dtype=np.float32
)

left_RT = np.array(
    [
        [0.88649035, -0.46274707, -0.00, -14.42428],
        [-0.070794605, -0.13562201, -0.98822814, 86.532959],
        [0.45729965, 0.8760547, -0.1529876, 235.35446],
    ],
    dtype=np.float32,
)

right_K = np.array(
    [[1100.0, 0.0, 360.0], [0.0, 1100.0, 640.0], [0.0, 0.0, 1.0]], dtype=np.float32
)

right_RT = np.array(
    [
        [0.98480779, -0.17364818, -4.9342116e-8, -0.98420829],
        [-0.026566068, -0.15066338, -0.98822814, 85.070221],
        [0.17160401, 0.97321475, -0.1529876, 236.97873],
    ],
    dtype=np.float32,
)

F = np.array(
    [
        [3.283965767647195e-7, -6.76098398189792e-6, 0.0021123144539793737],
        [-8.046341661808292e-6, 3.05632173594769e-8, 0.05124913199417346],
        [0.0048160232373805345, -0.051062699158041805, 1.0706286680443888],
    ],
    dtype=np.float32,
)

In [3]:
def split_image(img: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    cols= np.shape(img)[1]
    img_left = img[:, : cols // 2, :]
    img_right = img[:, cols // 2 :, :]
    return img_left, img_right


def show_img(img: np.ndarray):
    if len(img.shape) > 2:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img[:, :])
    plt.tight_layout()
    plt.axis("off")
    plt.show()

In [4]:
def find_points(
    img: np.ndarray, draw_points: bool = False, hmg: bool = False
) -> np.ndarray:
    points = []
    rows = img.shape[0]
    for row in range(rows):
        cur_row = img[row, :, 2]
        cur_row[np.where(cur_row < 5)] = 0
        idx = np.argmax(cur_row)
        if idx == 0:
            continue
        if hmg:
            points.append((idx, row, 1))
        else:
            points.append((idx, row))
        if draw_points:
            img[row, idx] = (0, 255, 0)
    return np.array(points)


In [5]:
def frame_differencing(image_sequence: list[str]) -> list[np.ndarray]:
    frames = []
    for i in range(1, len(image_sequence)):
        prev = cv2.imread(image_sequence[i - 1])
        cur = cv2.imread(image_sequence[i])
        difference = cv2.absdiff(cur, prev)
        difference = cv2.GaussianBlur(difference, ksize=(21, 21), sigmaX=5.0)
        frames.append(difference)
    return frames

In [6]:
def find_right_fg_mask() -> np.ndarray:
    image = cv2.imread('./SBS images/000.jpg')
    _ , image = split_image(img=image)
    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (200, 0, 560, 890) 
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    return mask2


def find_left_fg_mask() -> np.ndarray:
    image = cv2.imread('./SBS images/000.jpg')
    image, _ = split_image(img=image)
    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (155, 50, 345, 830)  
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8') 
    return mask2

In [7]:
def fg_mask() -> np.ndarray:
    image = cv2.imread('./SBS images/000.jpg')

    # Initialize the mask
    mask = np.zeros(image.shape[:2], np.uint8)

    # Background and foreground models
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Define the rectangle (x, y, width, height)
    rect = (155, 0, 1125, 890)

    image[0:938, 500:928] = [0,0,0]
    image[0:275, 392:716] = [0,0,0]

    # Apply the GrabCut algorithm
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 20, cv2.GC_INIT_WITH_RECT)

    # Modify the mask
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    mask2[0:938, 500:928] = 0
    mask2[0:275, 392:716] = 0
    return mask2

In [8]:
def find_corresponding_points(
    left_points: np.ndarray, right_points: np.ndarray, lines: np.ndarray
) -> tuple[np.ndarray, np.ndarray]:
    left_cor = []
    right_cor = []
    for line in lines:
        a, b, c = line
        if b == 0:
            continue

        min_err = 99999999
        cor_pr = None
        cor_pl = None

        for pl, pr in zip(left_points, right_points):
            x, y = pr[:2]
            er = abs(a * x + b * y + c)

            if er < min_err:
                min_err = er
                cor_pr = pr
                cor_pl = pl
        left_cor.append(cor_pl)
        right_cor.append(cor_pr)
    return left_cor, right_cor

In [9]:
def find_corresponding_point(
    line: np.ndarray, right_points: np.ndarray
) -> np.ndarray:
    a, b, c = line
    if b == 0: return np.array[[0,0,0]]
    min_err = 99999999
    cor_pr = None

    for pr in right_points:
        x, y = pr[:2]
        er = abs(a * x + b * y + c)

        if er < min_err:
            min_err = er
            cor_pr = pr
    return cor_pr

In [10]:
frames = glob("./SBS images/*.jpg")
frames = natsorted(frames, reverse=False)
frames = frame_differencing(frames)

In [11]:
mask = fg_mask()

In [12]:
P1 = left_K @ left_RT
P2 = right_K @ right_RT
P1, P2

(array([[ 1.0511182e+03, -1.4736737e+02, -5.5075535e+01,  7.0303328e+04],
        [ 2.2187718e+02,  4.2505298e+02, -1.0861403e+03,  2.3715981e+05],
        [ 4.5729965e-01,  8.7605470e-01, -1.5298760e-01,  2.3535446e+02]],
       dtype=float32),
 array([[ 1.1450660e+03,  1.5934430e+02, -5.5075588e+01,  8.4229711e+04],
        [ 8.0603897e+01,  4.5712772e+02, -1.1849630e+03,  2.4524362e+05],
        [ 1.7160401e-01,  9.7321475e-01, -1.5298760e-01,  2.3697873e+02]],
       dtype=float32))

In [13]:
points_3d = []
for frame in frames:
    img = frame.copy()
    img = img * mask[:, :, np.newaxis]
    
    left, right = split_image(img=img)
    left_points = find_points(img=left, draw_points=True, hmg=True)
    right_points = find_points(img=right, draw_points=True, hmg=True)    
    
    lines = (F@left_points.T).T
    
    right_cor = []
    for line in lines:
        right_cor.append(find_corresponding_point(line=line, right_points=right_points))
    right_cor = np.array(right_cor)
    
    
    for pl, pr in zip(left_points, right_cor):        
        u1, v1 = pl[:2]
        u2, v2 = pr[:2]
        m = np.vstack(
            [
                u1 * P1[2] - P1[0],
                v1 * P1[2] - P1[1],
                u2 * P2[2] - P2[0],
                v2 * P2[2] - P2[1],
            ],
            dtype=np.float32,
        )
        _, _, Vt = np.linalg.svd(m)
        X = Vt[-1]
        X = X / X[3]
        error = m@X
        error = np.linalg.norm(error)
        if (error < 20): 
            points_3d.append(X)
            print(X)
    

[-29.5757   -21.15969  114.013016   1.      ]
[-29.777632 -21.759727 113.85657    1.      ]
[-29.556442 -20.483427 113.62988    1.      ]
[-29.759184 -21.087866 113.47409    1.      ]
[-29.3636   -19.866175 113.24417    1.      ]
[-29.35589  -18.444517 110.72164    1.      ]
[-29.171757 -19.945381 114.14827    1.      ]
[-29.5757   -21.15969  114.013016   1.      ]
[-29.17912 -19.9393  113.78685   1.     ]
[-29.383625 -20.547123 113.63005    1.      ]
[-29.15826  -19.253862 113.40031    1.      ]
[-29.3636   -19.866175 113.24417    1.      ]
[-29.148664 -17.822058 110.87111    1.      ]
[-29.15613  -17.815182 110.504      1.      ]
[-29.391098 -19.123812 110.39097    1.      ]
[-29.36716  -18.434565 110.17259    1.      ]
[-28.969688 -19.330866 114.12594    1.      ]
[-29.376217 -20.552988 113.99038    1.      ]
[-29.17912 -19.9393  113.78685   1.     ]
[-29.18281  -19.936281 113.60607    1.      ]
[-28.984365 -19.318384 113.4005     1.      ]
[-29.21825  -20.605392 113.269714   1.    

In [14]:
def project_3d_to_2d(point_3d, projection_matrix):
    point_2d = np.dot(projection_matrix, point_3d)
    points_2d = point_2d / point_2d[-1]
    return points_2d

In [15]:
colored_img = cv2.imread('./SBS images/000.jpg')
_, colored_img = split_image(img=colored_img)
data_points = []
for point in points_3d:
    point_2d = project_3d_to_2d(point_3d=point, projection_matrix=P2)
    u, v = map(int, point_2d[:2])
    
    if 0 <= u < colored_img.shape[1] and 0 <= v < colored_img.shape[0]: 
        color = colored_img[v, u]
        data_points.append([*point[:-1], color[2], color[1], color[0]])

In [16]:
with open('./F11115103.txt', 'w') as file:
    for cord in data_points:
        x, y, z, r, g, b= cord

        if abs(x) > 33: continue
        if abs(y) > 40: continue
        if abs(z) > 220: continue

        file.write('{:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(x, y, z, r, g, b))